In [137]:
%reset -sf

In [138]:
# метод 0 - используем networkx для подсчета компонент
import networkx as nx
def method0(nv, edges):
    g = nx.Graph()
    g.add_nodes_from(range(nv))
    g.add_edges_from(edges)
    verts = [list(adj.keys()) for node, adj in g.adjacency()]
    nc = len(list(nx.connected_components(g)))
    return nc, verts

In [203]:
# метод 1 - вызываем тестируемую программу
def method1(nv, edges):
    rdata = [(nv, len(edges))] + [(a+1,b+1) for a,b in edges]
    sdata = '\n'.join('%s %s' % (a,b) for a, b in rdata) + '\n'
    pname = 'count-comps.py'
    from sys import executable
    from subprocess import run, STDOUT, PIPE
    res = run([executable, pname], stderr=STDOUT, stdout=PIPE, input=bytes(sdata,'utf8'))
    nc = int(res.stdout.strip().split()[0])
    return nc, str(res.stdout, 'utf8')

In [204]:
# метод 2 - dfs()
def method2(nv, edges):
    verts = [[] for _ in range(nv)]
    for a, b in edges:
        verts[a].append(b)
        verts[b].append(a)

    visited = [False] * nv
    nc = 0
        
    def dfs(v):
        visited[v] = True
        for w in verts[v]:
            if not visited[w]:
                dfs(w)

    for i in range(len(visited)):
        if not visited[i]:
            nc += 1
            dfs(i)
    return nc, verts

In [205]:
sdata1 = '''4 2
1 2
3 2
'''
sdata2 = '''4 3
1 2
3 2
4 3
'''

sdata3 = '''6 0
6 6
1 1
2 2
3 3
4 4
5 1
1 5
3 2
4 3
2 1
'''

sdata4 = '''0 0
1 1
1 1
1 1
1 1
'''

In [206]:
# проверка на небольшом наборе тестовых данных
for case, sdata in enumerate([sdata1, sdata2, sdata3, sdata4]):
    rdata = [[int(x) for x in s.split()] for s in sdata.splitlines()]
    nv, ne = rdata[0]
    edges = rdata[1:]

    ne = len(edges)
    if edges and nv <= 0:
        nv = max([max(a,b) for a,b in edges])
    assert(all([1<=a<=nv and 1<=b<=nv for a,b in edges]))
    edges = [(a-1, b-1) for a,b in edges]

    nc0, _ = method0(nv, edges)
    nc1, _ = method1(nv, edges)
    assert nc1 == nc0
    nc2, _ = method2(nv, edges)
    assert nc2 == nc0
print('ok')

ok


In [209]:
# проверка на большом массиве случайных графов
import networkx as nx
from random import randint
max_nodes = 100
max_graphs = 1000
sub_program = 0
rand_graphs = [nx.gnm_random_graph(randint(1,max_nodes),randint(1,max_nodes))
               for _ in range(max_graphs)]

In [210]:
for case, g in enumerate(rand_graphs):
    nv = len(g.nodes)
    edges = list(g.edges)
    nc0, verts0 = method0(nv, edges)
    if sub_program:
        nc1, verts1 = method1(nv, edges)
        assert nc1 == nc0
    nc2, verts2 = method2(nv, edges)
    assert nc2 == nc0
print('ok')

ok
